# 图像翻译（Image Translation） 之 CycleGAN

在 MMGeneration中调用 Image Translation 预训练模型

> 作者：[同济子豪兄](https://space.bilibili.com/1900783)、杨逸飞 2022-4-10

## 进入MMGeneration主目录

In [1]:
import os
os.chdir('mmgeneration')

## 导入工具包

In [2]:
# 导入mmcv和mmgeneration
import mmcv
from mmgen.apis import init_model, sample_img2img_model

# 导入 opencv
import cv2

# 导入numpy和matplotlib
import numpy as np
import matplotlib.pyplot as plt

# 定义可视化图像函数，输入图像 array，可视化图像
def show_img_from_path(img_path):
    '''输入 array，matplotlib 可视化格式为 RGB，因此需将 BGR 转 RGB，最后可视化出来'''
    img_BGR = cv2.imread(img_path)
    img_RGB = cv2.cvtColor(img_BGR, cv2.COLOR_BGR2RGB)
    plt.imshow(img_RGB)
    plt.show()

## 封装函数，输入图像路径，保存输出图像并可视化

In [3]:
def gen_vis_fake_img(input_path, output_path='outputs/F1/', target_domain='winter', figsize=15, frame_id=0):
    
    # 读入输入图像，获取高宽尺寸
    input_img = cv2.imread(input_path)

    # 生成另外一个图像域的图像，注意 target_domain 要设置正确
    fake_imgs = sample_img2img_model(model, input_path, target_domain=target_domain)
    # 获取生成图像尺寸
    img_size = fake_imgs.shape[2]

    # 分别抽取RGB三通道图像
    RGB = np.zeros((img_size, img_size, 3))
    RGB[:,:,0] = fake_imgs[0][2]
    RGB[:,:,1] = fake_imgs[0][1]
    RGB[:,:,2] = fake_imgs[0][0]
    # 将生成图转为输入图像大小
    RGB = cv2.resize(RGB, dsize=(input_img.shape[1],input_img.shape[0]))
    # 像素值归一化为0-255的uint8自然图像
    RGB = 255 * (RGB - RGB.min()) / (RGB.max()-RGB.min())
    # 像素值转为整数
    RGB = RGB.astype('uint8')
    # 导出生成的图像文件
    # cv2.imwrite(output_path+target_domain+'_'+input_path.split('/')[-1], cv2.cvtColor(RGB, cv2.COLOR_BGR2RGB))

    plt.figure(figsize=(figsize, figsize))
    # 展示原始输入图像
    plt.subplot(1,2,1)
    plt.title('input')
    input_RGB = cv2.cvtColor(input_img, cv2.COLOR_BGR2RGB)
    plt.xticks([])
    plt.yticks([])
    plt.imshow(input_RGB)

    # 展示生成图
    plt.subplot(1,2,2)
    plt.title(target_domain)
    plt.xticks([])
    plt.yticks([])
    plt.imshow(RGB)

    # plt.show()
    plt.savefig(f"{temp_dir}/{frame_id:06d}.jpg")

## 冬天和夏天互转，导入模型

In [4]:
# 指定config文件路径
config_file = 'configs/cyclegan/cyclegan_lsgan_resnet_in_summer2winter_b1x1_250k.py'
# 指定预训练模型权重文件路径
checkpoint_file = 'https://download.openmmlab.com/mmgen/cyclegan/refactor/cyclegan_lsgan_resnet_in_1x1_246200_summer2winter_convert-bgr_20210902_165932-fcf08dc1.pth'

# 输出图像宽高像素尺寸
img_size = 256

model = init_model(config_file, checkpoint_file, device='cuda:0')

/environment/miniconda3/lib/python3.7/site-packages/mmcv/cnn/bricks/conv_module.py:151: UserWarning: Unnecessary conv bias before batch/instance norm
  'Unnecessary conv bias before batch/instance norm')


load checkpoint from http path: https://download.openmmlab.com/mmgen/cyclegan/refactor/cyclegan_lsgan_resnet_in_1x1_246200_summer2winter_convert-bgr_20210902_165932-fcf08dc1.pth


## 生成视频

In [ ]:
# 创建存放图像的临时文件夹
import time
temp_dir = time.strftime('%Y%m%d%H%M%S')

if os.path.exists(temp_dir):
    pass
else:
    os.mkdir(temp_dir)
    print('创建文件夹 {} 用于存放每帧图像'.format(temp_dir))
    
input_video = 'data/tongji_video.mp4'

# 读入待预测视频
imgs = mmcv.VideoReader(input_video)
prog_bar = mmcv.ProgressBar(len(imgs))

# 对视频逐帧处理
for frame_id, img in enumerate(imgs):
    cv2.imwrite('temp.jpg', img)
    gen_vis_fake_img('temp.jpg', target_domain='winter', figsize=15, frame_id=frame_id)
    # 更新进度条
    prog_bar.update()
    
# 由每一帧的图像文件，生成视频
mmcv.frames2video(temp_dir, 'outputs/F2/F2_output.mp4', fps=imgs.fps, fourcc='mp4v')

创建文件夹 20220508154035 用于存放每帧图像
[>>                             ] 20/295, 1.1 task/s, elapsed: 17s, ETA:   240s

/environment/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:25: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).


[>>>>>>>>>>>>>>>>>>>>>>>>>>>> ] 289/295, 1.2 task/s, elapsed: 250s, ETA:     5s